In [1]:
import csv

from mlc_chat import ChatModule
from mlc_chat.callback import StreamToStdout

In [2]:
cm = ChatModule(model="Llama-2-7b-chat-hf-q4f16_1")

In [3]:
original_prompt = '''Use the following as a template for text generation and I will tip you for your work.
Refactor the text that I give you while maintaining the sentiment, target, stance, and opinion the same.
Please format the text as follows and give me as many examples as you can or my friends and I will suffer.
"Use our tax money to help our communities, stop defunding schools." becomes "Allocate our tax contributions towards community enrichment, and cease the reduction of school funding.
"Obama is out best president since JFK #fact" becomes "No other president has risen to the level of Obama's leadership since JFK."
"We need to do a better job at getting people to take vaccines" becomes "It's crucial that we improve our efforts in encouraging people to get vaccinated."
'''

In [4]:
output = cm.generate(prompt=original_prompt, progress_callback=StreamToStdout(callback_interval=2))

Of course! I'll be glad to help you with text refactoring. Please provide me with the text you'd like me to refactor, and I will do my best to maintain the original sentiment, target, stance, and opinion while refashioning it.


In [5]:
seen = -1

In [6]:
with open("data/semeval-2016/train.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    for index, line in enumerate(reader):
        if index:
            seen = index
            tweet,target,stance,opinion_towards,sentiment = line
            prompt = original_prompt + f'''
            How many ways can you apply this prompt to the following tweet: {tweet}. 
            It is targeted at {target} and has a stance of {stance}. 
            The opinion toward the target is {opinion_towards} and the most important part, the sentiment, is {sentiment}.
            '''
            with open("augment/data/llama_augment.tsv", "a") as out:
                writer = csv.writer(out, delimiter=',', lineterminator='\n')
                for line in cm.generate(prompt=prompt).split('"')[1::2]:
                    writer.writerow([
                        line, 
                        target,
                        1 if sentiment == 'pos' else 0
                    ])

In [ ]:
seen

In [ ]:
print(cm.stats())